<a href="https://colab.research.google.com/github/JandreiSS/analise-fundamentalista/blob/master/An%C3%A1lise_Fundamentalista.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import string
import requests
import warnings
warnings.filterwarnings('ignore')

In [2]:
url = 'http://fundamentus.com.br/resultado.php'

In [3]:
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

In [4]:
r = requests.get(url, headers=header)

In [5]:
dflist = pd.read_html(r.text, thousands='.', decimal=',')[0]

In [6]:
for coluna in ['Div.Yield', 'Mrg Ebit', 'Mrg. Líq.', 'ROIC', 'ROE', 'Cresc. Rec.5a']:
  dflist[coluna] = dflist[coluna].str.replace('.', '')
  dflist[coluna] = dflist[coluna].str.replace(',', '.')
  dflist[coluna] = dflist[coluna].str.rstrip('%').astype('float') / 100

In [7]:
# dflist['Papel'].values

#<strong>Analisando Dados</strong>

In [8]:
df = dflist[dflist['Liq.2meses'] > 1000000]

<p><strong>Enterprise Value / Earnings Before Interest and Taxes</strong></p>
<p>(Valor do Negócio / Lucro Antes dos Juros e Taxas)</p>

In [9]:
ranking = pd.DataFrame()
ranking['Position'] = range(1,151)
ranking['EV/EBIT'] = df[df['EV/EBIT'] > 0].sort_values(by=['EV/EBIT'])['Papel'][:150].values
ranking['ROIC'] = df.sort_values(by=['ROIC'], ascending=False)['Papel'][:150].values

In [10]:
ranking

,Position,EV/EBIT,ROIC
0,1,PSSA3,WIZS3
1,2,GPIV33,PSSA3
2,3,SMLS3,TOTS3
3,4,WIZS3,ODPV3
4,5,ENAT3,SMLS3
...,...,...,...
145,146,RADL3,JHSF3
146,147,GNDI3,CAML3
147,148,PTBL3,BRKM5
148,149,LOGG3,LOGG3


Cálculo utilizando o rankeamento pela Fórmula Mágica: Buscar o papel, pegando a posição que ele tem no EV/EBIT e somando a posição que ele tem no ROIC. Quanto menor, melhor

In [11]:
auxA = ranking.pivot_table(columns='EV/EBIT', values='Position')
auxB = ranking.pivot_table(columns='ROIC', values='Position')
t = pd.concat([auxA, auxB])

In [12]:
t

,AALR3,ABEV3,AGRO3,ALPA4,ALSO3,ALUP11,AMAR3,ANIM3,ARZZ3,B3SA3,BEEF3,BRDT3,BRFS3,BRKM5,BRML3,BRPR3,CAMB3,CAML3,CARD3,CCPR3,CCRO3,CEAB3,CESP6,CIEL3,CLSC4,CMIG3,CMIG4,COCE5,COGN3,CPFE3,CPLE3,CPLE6,CRFB3,CSAN3,CSMG3,CSNA3,CVCB3,CYRE3,DIRR3,DTEX3,...,SAPR4,SBSP3,SEER3,SGPS3,SHUL4,SLCE3,SMLS3,SMTO3,SULA11,SUZB3,TASA4,TEND3,TESA3,TGMA3,TIET11,TIET4,TIMP3,TOTS3,TRIS3,TRPL4,TUPY3,UGPA3,UNIP6,USIM3,USIM5,VALE3,VIVT3,VIVT4,VLID3,VULC3,VVAR3,WIZS3,WSON33,YDUQ3,EZTC3,LWSA3,MGLU3,PNVL3,PRNR3,WEGE3
Position,114.0,84.0,68.0,139.0,82.0,6.0,50.0,123.0,140.0,138.0,28.0,109.0,44.0,132.0,69.0,121.0,52.0,144.0,72.0,78.0,67.0,83.0,103.0,31.0,30.0,14.0,12.0,56.0,142.0,48.0,17.0,21.0,61.0,125.0,26.0,43.0,13.0,133.0,75.0,135.0,...,10.0,38.0,37.0,105.0,90.0,46.0,3.0,87.0,8.0,143.0,99.0,47.0,80.0,42.0,65.0,63.0,57.0,89.0,97.0,24.0,29.0,110.0,33.0,62.0,58.0,7.0,73.0,74.0,35.0,55.0,141.0,4.0,27.0,81.0,NaN,NaN,NaN,NaN,NaN,NaN
Position,141.0,15.0,84.0,20.0,131.0,29.0,52.0,116.0,30.0,34.0,9.0,98.0,70.0,148.0,127.0,NaN,57.0,147.0,46.0,115.0,42.0,105.0,136.0,35.0,124.0,66.0,67.0,96.0,NaN,37.0,62.0,63.0,50.0,114.0,41.0,56.0,40.0,130.0,122.0,126.0,...,13.0,36.0,72.0,NaN,90.0,73.0,5.0,88.0,102.0,145.0,87.0,25.0,139.0,8.0,23.0,24.0,78.0,3.0,53.0,51.0,60.0,106.0,77.0,128.0,129.0,18.0,103.0,104.0,95.0,65.0,110.0,1.0,28.0,17.0,117.0,99.0,107.0,68.0,140.0,11.0


In [13]:
rank = t.dropna(axis=1).sum().sort_values()
rank[:30]

PSSA3      3.0
WIZS3      5.0
SMLS3      8.0
SAPR11    21.0
SAPR4     23.0
SAPR3     25.0
VALE3     25.0
LEVE3     32.0
EQTL3     34.0
ALUP11    35.0
MRFG3     36.0
BEEF3     37.0
TGMA3     50.0
CVCB3     53.0
ENAT3     54.0
WSON33    55.0
PETR4     60.0
PETR3     64.0
CIEL3     66.0
QUAL3     66.0
CSMG3     67.0
RAPT4     69.0
TEND3     72.0
SBSP3     74.0
JBSS3     75.0
TRPL4     75.0
CMIG4     79.0
CPLE3     79.0
ENBR3     79.0
CMIG3     80.0
dtype: float64

In [14]:
# Cotação, P/L, EBITDA, Div.Yield, ROE, EV/EBIT
# Comparar papéis do mesmo setor
lista = df.loc[:, ['Papel', 'Cotação', 'P/L', 'Div.Yield', 'ROE', 'EV/EBIT']]

In [15]:
# lista

In [16]:
from concurrent.futures import ThreadPoolExecutor, as_completed

In [17]:
def get(url, timeout):
    return requests.get(url, headers=header, timeout = timeout)

In [18]:
def requestUrls(urls, timeout = 5):
    with ThreadPoolExecutor(max_workers = 5) as executor:
        agenda = { executor.submit(get, url, timeout): url for url in urls }

        for tarefa in as_completed(agenda):     
            try:
                conteudo = tarefa.result()
            except Exception as e:
                print ("Não foi possível fazer a requisição! \n{}".format(e))
            else:
                yield conteudo

In [19]:
urls = 'https://www.fundamentus.com.br/detalhes.php?papel=' + lista['Papel'].values
# urls = 'https://www.fundamentus.com.br/detalhes.php?papel=' + df['Papel'].values
# urls = 'https://www.fundamentus.com.br/detalhes.php?papel=' + dflist['Papel'].values

In [20]:
# dflist['Papel'].values

In [21]:
requisicoes = requestUrls(urls, timeout=7)

In [22]:
lista_empresas = []

In [23]:
for requisicao in requisicoes:
    codigo = requisicao.status_code
    url = requisicao.url
    conteudo = requisicao.content
    lista_empresas += [conteudo]
    # print ("{}: {}".format(codigo, url))

In [24]:
# type(lista_empresas)

In [25]:
# lista_empresas

In [26]:
# conteudo = pd.read_html(lista_empresas[0])

In [27]:
html_data = []

In [28]:
# Papel que quer pesquisar
papel = ['WEGE3', 'USIM5']

In [29]:
from bs4 import BeautifulSoup

In [30]:
def readHTML(content):
  soup = BeautifulSoup(content, 'html.parser')
  for tr in soup.find_all('tr'):
    tds = soup.find_all('td')
    papel_html = tds[1].text
    cotacao_html = tds[3].text
    empresa_html = tds[9].text
    setor_html = tds[13].text
    subsetor_html = tds[17].html
    valor_firma_html = tds[25].text
    ev_ebitda_html = tds[81].text
    # print(' \
    #   Papel: {}, \
    #   Cotação: {}, \
    #   Empresa: {}, \
    #   Setor: {}, \
    #   Subsetor: {}, \
    #   Valor da firma: {}, \
    #   EV/EBITDA: {}'.format(papel_html, cotacao_html, empresa_html, setor_html, subsetor_html, valor_firma_html, ev_ebitda_html))
    mid_string = (' \
      Papel: {}, \
      Cotação: {}, \
      Empresa: {}, \
      Setor: {}, \
      Subsetor: {}, \
      Valor da firma: {}, \
      EV/EBITDA: {}'.format(papel_html, cotacao_html, empresa_html, setor_html, subsetor_html, valor_firma_html, ev_ebitda_html))
    complete_string = '{' + mid_string + '}'
    print(complete_string)
    break

In [31]:
for i in range(len(lista_empresas)):
  ticket = pd.read_html(lista_empresas[i], thousands='.', decimal=',')[0][1][0]
  for j in range(len(papel)):
    if ticket == papel[j]:
      conteudo = lista_empresas[i]
      html_data = [conteudo]
      readHTML(conteudo)
      # print ('{}'.format(conteudo))

{       Papel: USIM5,       Cotação: 7,52,       Empresa: USIMINAS PNA N1,       Setor: Siderurgia e Metalurgia,       Subsetor: None,       Valor da firma: 12.975.900.000,       EV/EBITDA: 
          6,08}
{       Papel: WEGE3,       Cotação: 53,99,       Empresa: WEG SA ON N1,       Setor: Máquinas e Equipamentos,       Subsetor: None,       Valor da firma: 112.871.000.000,       EV/EBITDA: 
          41,52}
